In [1]:
import torch.utils.data as data

import random
import numbers
from PIL import Image, ImageMath
import os
import os.path
import numpy as np
import struct
import math

import torch
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import faiss
import time

import pcl
from PIL import Image, ImageDraw
import faiss

import timeit
from pyquaternion import Quaternion

%matplotlib qt5

In [2]:
class KNNBuilder_GPU:
    def __init__(self, k):
        self.k = k
        self.dimension = 3
        
        # we need only a StandardGpuResources per GPU
        self.res = faiss.StandardGpuResources()
        self.res.setTempMemoryFraction(0.1)
        self.flat_config = faiss.GpuIndexFlatConfig()
        self.flat_config.device = 0
        
    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.GpuIndexFlatL2(self.res, self.dimension, self.flat_config)  # dimension is 3
        index.add(database)
        return index
    
    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I
    
    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I
    

class KNNBuilder:
    def __init__(self, k):
        self.k = k
        self.dimension = 3

    def build_nn_index(self, database):
        '''
        :param database: numpy array of Nx3
        :return: Faiss index, in CPU
        '''
        index = faiss.IndexFlatL2(self.dimension)  # dimension is 3
        index.add(database)
        return index

    def search_nn(self, index, query, k):
        '''
        :param index: Faiss index
        :param query: numpy array of Nx3
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        D, I = index.search(query, k)
        return D, I

    def self_build_search(self, x):
        '''

        :param x: numpy array of Nxd
        :return: D: numpy array of Nxk
                 I: numpy array of Nxk
        '''
        x = np.ascontiguousarray(x, dtype=np.float32)
        index = self.build_nn_index(x)
        D, I = self.search_nn(index, x, self.k)
        return D, I
    

class PCSampler:
    def __init__(self, leaf_size, minimum_pc_num):       
        self.leaf_size = leaf_size
        self.minimum_pc_num = minimum_pc_num
    
    def sample_pc(self, pc, leaf_size):
        '''
        :param pc: input numpy array of Nx3
        :return: sampled_pc of Mx3
        '''
        cloud = pcl.PointCloud(pc)
        sor = cloud.make_voxel_grid_filter()
        sor.set_leaf_size(leaf_size, leaf_size, leaf_size)
        cloud_filtered = sor.filter()
        sampled_pc = np.asarray(cloud_filtered)
        
        return sampled_pc
    
    def sample_pc_wrapper(self, pc):
        '''
        ensure that the sampled pc is more than a certain amount
        '''
        retry_counter = 0
        
        sampled_pc = self.sample_pc(pc, self.leaf_size)
        while sampled_pc.shape[0] < self.minimum_pc_num:
            retry_counter += 1
            leaf_size = self.leaf_size - 0.04*retry_counter
            if leaf_size <= 0:
                break
            sampled_pc = self.sample_pc(pc, leaf_size)
        
        return sampled_pc
    
    
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)
        

In [3]:
def load_bin(file_path, num_cols=6):
    pc_np = np.fromfile(file_path, dtype=np.float32)
    pc_np = np.reshape(pc_np, (-1, num_cols))
    
    # remove the surface normals, I will compute it myself using python-pcl
    return pc_np[:, 0:3]


def load_test_gt_txt(txt_path):
    f = open(txt_path, 'r')
    lines_list = f.readlines()
    
    sample_num = 828  # first line is header
    pos_idx_list = []  # a list that stores the nearby sample indexes for each sample
    for i in range(sample_num):
        pos_idx_list.append([])
    
    dataset = []
    for i, line_str in enumerate(lines_list):
        if i == 0:
            # skip the headline
            continue
        # convert each line to a dict
        line_splitted_list = line_str.split()
        try:
            assert len(line_splitted_list) == 11
        except Exception:
            print('Invalid line.')
            print(i)
            print(line_splitted_list)
            continue
        
        anc_idx = int(line_splitted_list[0].strip())
        pos_idx = int(line_splitted_list[1].strip())
        t = [float(line_splitted_list[4].strip()), 
             float(line_splitted_list[5].strip()), 
             float(line_splitted_list[6].strip())]  # tx, ty, tz
        q = [float(line_splitted_list[7].strip()), 
             float(line_splitted_list[8].strip()), 
             float(line_splitted_list[9].strip()),
             float(line_splitted_list[10].strip())]  # quaternion: w, x, y, z
        
#         print(anc_idx)
#         print(pos_idx)
#         print(t)
#         print(q)

        data = {'anc_idx': anc_idx, 'pos_idx': pos_idx, 't': t, 'q': q}
        dataset.append(data)
        
        
        # build negative sample
        pos_idx_list[anc_idx].append(pos_idx)
        pos_idx_list[pos_idx].append(anc_idx)
        
        
    f.close()
    
#     print(pos_idx_list)
    
    
    # build negative sample
    # set random seed to ensure the same result
    random.seed(1)
    for i, data in enumerate(dataset):
        anc_idx = data['anc_idx']
        
        while True:
            rand_idx = random.randint(0, sample_num-1)  # a<=x<=b
            if not (rand_idx in pos_idx_list[anc_idx]):
                data['neg_idx'] = rand_idx
                break
    
    # reset reandom seed
    random.seed()
    
    return dataset

In [ ]:
dataset = load_test_gt_txt('/ssd/dataset/oxford/test_models_20k/groundtruths.txt')
print(dataset)

In [4]:
# get surface normal
def Surface_normals(cloud):
    ne = cloud.make_NormalEstimation()
    tree = cloud.make_kdtree()
    ne.set_SearchMethod(tree)
#     ne.set_RadiusSearch(2)
    ne.set_KSearch(9)
    cloud_normals = ne.compute()
    return cloud_normals

# 1. voxel grid filer
# 2. apply outlier filter
# 3. calculate surface normal and curvature
def process_pc(pc_np):
#     print('origin')
#     print(pc_np.shape)
    
    # 1. voxel grid filer
    sampler = PCSampler(leaf_size=0.2, minimum_pc_num=20480)
    pc_np = sampler.sample_pc_wrapper(pc_np)
#     print('after voxel grid filter')
#     print(pc_np.shape)
    
    # 2. apply outlier filter
    fil = pcl.PointCloud(pc_np).make_statistical_outlier_filter()
    fil.set_mean_k(50)
    fil.set_std_dev_mul_thresh(2)
    pc_np = np.asarray(fil.filter())
#     print('after outlier filter')
#     print(pc_np.shape)
    
#     fig = plt.figure()
#     ax = Axes3D(fig)
#     ax.scatter(pc_np[:,0].tolist(), pc_np[:,1].tolist(), pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
#     axisEqual3D(ax)

#     plt.ion()
#     plt.show()
    
    # 3. calculate surface normal and curvature
    assert pc_np.shape[1] == 3
    cloud = pcl.PointCloud(pc_np)    
    sn = Surface_normals(cloud)
    sn_np = np.asarray(sn.to_array(), dtype=np.float32)  # Nx4, nx,ny,nz,curvature
    
    output_np = np.concatenate((pc_np, sn_np), axis=1)  # Nx7
    
    return output_np






# convert bin to npy
bin_folder = '/ssd/dataset/oxford/test_models_20k'
npy_folder = '/ssd/dataset/oxford/test_models_20k_np'

min_pc_num = 1000000
content = os.listdir(bin_folder)
for i, filename in enumerate(content):
    if not ('.bin' in filename):
        continue
    
    # load bin
    pc_np = load_bin(os.path.join(bin_folder, filename))
    
    # process pc
    pc_np = process_pc(pc_np)

    # save pc as numpy array
    npy_file_name = filename[0:-3] + 'npy'
    np.save(os.path.join(npy_folder, npy_file_name), pc_np)

    if pc_np.shape[0] < min_pc_num:
        min_pc_num = pc_np.shape[0]
        
    if i % 100 == 0:
        print('%d - min_pc_num %d' % (i, min_pc_num))

print('minimum point number: %d' % min_pc_num)

0 - min_pc_num 19062
100 - min_pc_num 18905
200 - min_pc_num 18905
300 - min_pc_num 18905
400 - min_pc_num 18905
500 - min_pc_num 18905
600 - min_pc_num 18905
700 - min_pc_num 18905
800 - min_pc_num 18905
minimum point number: 18905


In [ ]:
# test registration
folder = '/ssd/dataset/oxford/test_models_20k_np'
dataset = load_test_gt_txt('/ssd/dataset/oxford/test_models_20k_np/groundtruths.txt')
print('length of dataset: %d' % len(dataset))
for i, data in enumerate(dataset):
    if i<80:
        continue
    
    anc_idx = data['anc_idx']
    pos_idx = data['pos_idx']
    neg_idx = data['neg_idx']
    t = np.asarray(data['t'], dtype=np.float32)
    q_list = data['q']
    q = Quaternion(np.asarray(q_list))
    
    T = q.transformation_matrix
    
    T[0, 3] = t[0]
    T[1, 3] = t[1]
    T[2, 3] = t[2]
    
    T = T[0:3, :]  # 3x4
    
    anc_pc_np = np.load(os.path.join(folder, '%d.npy'%anc_idx))[:, 0:3]
    pos_pc_np = np.load(os.path.join(folder, '%d.npy'%pos_idx))[:, 0:3]
#     anc_pc_np = load_bin(os.path.join('/ssd/dataset/oxford/test_models_16384', '%d.bin'%anc_idx))[:, 0:3]
#     pos_pc_np = load_bin(os.path.join('/ssd/dataset/oxford/test_models_16384', '%d.bin'%pos_idx))[:, 0:3]
    
    N = pos_pc_np.shape[0]
    pos_pc_np_pad = np.concatenate((pos_pc_np, np.ones((N, 1))), axis=1)
    print(pos_pc_np_pad.shape)
    
    tmp_pc_np = np.transpose(np.dot(T, np.transpose(pos_pc_np_pad)))
    print(tmp_pc_np.shape)
    
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(anc_pc_np[:,0].tolist(), anc_pc_np[:,1].tolist(), anc_pc_np[:,2].tolist(), s=0.1, c=[0.5,0.5,0.5])
    ax.scatter(tmp_pc_np[:,0].tolist(), tmp_pc_np[:,1].tolist(), tmp_pc_np[:,2].tolist(), s=0.1, c=[1, 0, 0])
#     ax.scatter(pos_pc_np[:,0].tolist(), pos_pc_np[:,1].tolist(), pos_pc_np[:,2].tolist(), s=0.1, c=[0, 0, 1])
    axisEqual3D(ax)

    plt.ion()
    plt.show()
    
    print('anc: %d, pos: %d, neg: %d' % (anc_idx, pos_idx, neg_idx))
    print(t)
    print(q)
    print(T)
    break

In [ ]:
# save txt to pickle
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)
    
save_obj(dataset, '/ssd/dataset/oxford/test_models_20k_np/groundtruths.pkl')
dataset_read = load_obj('/ssd/dataset/oxford/test_models_20k_np/groundtruths.pkl')
print(dataset_read)

In [ ]:
# convert txt to that used in descriptor training
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)
    
dataset = load_obj('/ssd/dataset/oxford/test_models_20k_np/groundtruths.pkl')
dataset_descriptor = []
dataset_descriptor_idx_set = set([])

for i, item in enumerate(dataset):   
    # process anc_idx
    anc_idx = item['anc_idx']
    if anc_idx not in dataset_descriptor_idx_set:
        nonneg_set = set([])
        nonneg_set.add(anc_idx)
        for j, item_j in enumerate(dataset):
            if item_j['anc_idx'] == anc_idx:
                nonneg_set.add(item_j['pos_idx'])
            if item_j['pos_idx'] == anc_idx:
                nonneg_set.add(item_j['anc_idx'])
        nonneg_idx_list = list(nonneg_set)
        dataset_descriptor.append({'anc_idx': anc_idx, 'pos_idx_list': nonneg_idx_list})
        dataset_descriptor_idx_set.add(anc_idx)
    
    # process pos_idx
    anc_idx = item['pos_idx']
    if anc_idx not in dataset_descriptor_idx_set:
        nonneg_set = set([])
        nonneg_set.add(anc_idx)
        for j, item_j in enumerate(dataset):
            if item_j['anc_idx'] == anc_idx:
                nonneg_set.add(item_j['pos_idx'])
            if item_j['pos_idx'] == anc_idx:
                nonneg_set.add(item_j['anc_idx'])
        nonneg_idx_list = list(nonneg_set)
        dataset_descriptor.append({'anc_idx': anc_idx, 'pos_idx_list': nonneg_idx_list})
        dataset_descriptor_idx_set.add(anc_idx)


    
# print(dataset_descriptor)
print(len(dataset_descriptor))
print(len(dataset_descriptor_idx_set))

# for i in range(828):
#     if i not in dataset_descriptor_idx_set:
#         print(i)
            
save_obj(dataset_descriptor, '/ssd/dataset/oxford/test_models_20k_np/gt_descriptor_testing.pkl')

In [ ]:
print(dataset_descriptor)